In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [26]:
!pip install dagshub mlflow
import dagshub
import mlflow
dagshub.init(repo_owner='TamariToradze', repo_name='ML-Final', mlflow=True)

Initialized MLflow to track repo "TamariToradze/ML-Final"

Repository TamariToradze/ML-Final initialized!

# Data Preprocessing

In [27]:
features = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip")
train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip")
stores = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")
test = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip")

In [28]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin


class Merger(BaseEstimator, TransformerMixin):
  
    def __init__(self, auxiliary_features, metadata_info):
       
        # Combine auxiliary data sources into unified reference dataset
        self.reference_data = auxiliary_features.merge(
            metadata_info, 
            how='inner', 
            on='Store'
        )
        # Ensure consistent datetime handling
        self.reference_data['Date'] = pd.to_datetime(self.reference_data['Date'])
    
    def fit(self, X, y=None):
        """
        Fit method required by sklearn interface.
        No actual fitting needed for this transformer.
        """
        return self
    
    def transform(self, X):
    
        # Create working copy to avoid modifying original data
        enhanced_data = X.copy()
        enhanced_data['Date'] = pd.to_datetime(enhanced_data['Date'])
        
        # Join with reference data using common keys
        enriched_dataset = enhanced_data.merge(
            self.reference_data,
            how='inner',
            on=['Store', 'Date', 'IsHoliday']
        )
        
        # Apply consistent ordering for reproducible results
        final_dataset = enriched_dataset.sort_values(
            by=['Date', 'Store', 'Dept']
        ).reset_index(drop=True)
        
        return final_dataset

In [29]:
class DateTimeFeatureExtractor(BaseEstimator, TransformerMixin):
    """Extracts basic datetime components and converts temperature units."""
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        data = X.copy()
        
        # Temperature conversion to Celsius
        if 'Temperature' in data.columns:
            data['Temperature'] = (data['Temperature'] - 32) * (5.0 / 9.0)
        
        # Date component extraction
        data['Day'] = data['Date'].dt.day
        data['Month'] = data['Date'].dt.month
        data['Year'] = data['Date'].dt.year
        
        return data

In [30]:
class HolidayFeatureGenerator(BaseEstimator, TransformerMixin):
    """Generates holiday-related features and proximity indicators."""
    
    def __init__(self):
        self.superbowl_dates = pd.to_datetime(['2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08'])
        self.laborday_dates = pd.to_datetime(['2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06'])
        self.thanksgiving_dates = pd.to_datetime(['2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29'])
        self.christmas_dates = pd.to_datetime(['2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27'])
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        data = X.copy()
        
        # ISO calendar components for holiday detection
        data['WeekNum'] = data['Date'].dt.isocalendar().week
        data['CalendarYear'] = data['Date'].dt.year
        
        # Holiday week detection function
        def check_holiday_week(dates, holiday_list):
            holiday_week_set = set((d.isocalendar().week, d.year) for d in holiday_list)
            return dates.apply(lambda d: (d.isocalendar().week, d.year) in holiday_week_set if pd.notnull(d) else False).astype(int)
        
        data['SuperbowlWeek'] = check_holiday_week(data['Date'], self.superbowl_dates)
        data['LaborDayWeek'] = check_holiday_week(data['Date'], self.laborday_dates)
        data['ThanksgivingWeek'] = check_holiday_week(data['Date'], self.thanksgiving_dates)
        data['ChristmasWeek'] = check_holiday_week(data['Date'], self.christmas_dates)
        
        # Holiday proximity calculations using fixed anchor dates
        thanksgiving_anchor = pd.to_datetime(data['Year'].astype(str) + "-11-24")
        christmas_anchor = pd.to_datetime(data['Year'].astype(str) + "-12-24")
        
        data['Days_to_Thanksgiving'] = (thanksgiving_anchor - data['Date']).dt.days
        data['Days_to_Christmas'] = (christmas_anchor - data['Date']).dt.days
        
        # Remove temporary columns
        data = data.drop(columns=['WeekNum', 'CalendarYear'])
        
        return data

In [31]:
class TemporalFeatureEngineer(BaseEstimator, TransformerMixin):
    """Combined transformer that applies both datetime and holiday feature extraction."""
    
    def __init__(self):
        self.datetime_extractor = DateTimeFeatureExtractor()
        self.holiday_generator = HolidayFeatureGenerator()
    
    def fit(self, X, y=None):
        self.datetime_extractor.fit(X, y)
        self.holiday_generator.fit(X, y)
        return self
    
    def transform(self, X):
        # Apply datetime features first
        data = self.datetime_extractor.transform(X)
        # Then apply holiday features
        data = self.holiday_generator.transform(data)
        return data

In [32]:
class NaFiller(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.promotional_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
        self.economic_cols = ['CPI', 'Unemployment']
        self.computed_means = {}
    
    def fit(self, X, y=None):
        for column in self.economic_cols:
            if column in X.columns:
                self.computed_means[column] = X[column].mean()
        return self
    
    def transform(self, X):
        data = X.copy()
        
        # Fill promotional markdowns with 0
        for column in self.promotional_cols:
            if column in data.columns:
                data[column] = data[column].fillna(0.0)
        
        # Fill economic indicators with computed mean
        for column in self.economic_cols:
            if column in data.columns and column in self.computed_means:
                data[column] = data[column].fillna(self.computed_means[column])
        
        return data

In [33]:
class CategoryMapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.store_type_map = {'A': 3, 'B': 2, 'C': 1}
        self.boolean_flag_map = {False: 0, True: 1}
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        data = X.copy()
        
        if 'Type' in data.columns:
            data['Type'] = data['Type'].map(self.store_type_map)
        
        if 'IsHoliday' in data.columns:
            data['IsHoliday'] = data['IsHoliday'].map(self.boolean_flag_map)
        
        return data

In [34]:
class StoreDataProcessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.processed_data = {}
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        self.processed_data = {}
        for store_id in X['Store'].unique():
            self.process_store_data(store_id, X)
        return self.processed_data
    
    def process_store_data(self, store_num, X):
        store_subset = X[X['Store'] == store_num].copy()
        
        # Detect if sales data is available (training vs test scenario)
        sales_available = 'Weekly_Sales' in store_subset.columns
        
        if sales_available:
            sum_fields = ['Weekly_Sales', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
        else:
            sum_fields = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
        
        first_fields = ['IsHoliday', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
                       'Type', 'Size', 'Day', 'Month', 'Year', 'SuperbowlWeek',
                       'LaborDayWeek', 'ThanksgivingWeek', 'ChristmasWeek',
                       'Days_to_Thanksgiving', 'Days_to_Christmas']
        
        aggregation_rules = {}
        
        # Add summation rules for existing columns
        for field in sum_fields:
            if field in store_subset.columns:
                aggregation_rules[field] = 'sum'
        
        # Add first-value rules for existing columns
        for field in first_fields:
            if field in store_subset.columns:
                aggregation_rules[field] = 'first'
        
        consolidated = store_subset.groupby(['Date', 'Store']).agg(aggregation_rules).reset_index()
        consolidated = consolidated.sort_values('Date').reset_index(drop=True)
        
        # Compute department proportions only when sales data exists
        if sales_available:
            dept_ratios = self.compute_dept_ratios(store_subset)
        else:
            dept_ratios = None
        
        self.processed_data[store_num] = (consolidated, dept_ratios)
        return consolidated
    
    def compute_dept_ratios(self, store_subset):
        dept_sales_totals = store_subset.groupby('Dept')['Weekly_Sales'].sum()
        overall_total = store_subset['Weekly_Sales'].sum()
        
        if overall_total == 0:
            dept_count = len(dept_sales_totals)
            return {dept: 1.0/dept_count for dept in dept_sales_totals.index}
        
        dept_ratios_dict = (dept_sales_totals / overall_total).to_dict()
        return dept_ratios_dict

In [35]:
from sklearn.pipeline import Pipeline


In [36]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('merge', Merger(features, stores)),
    ('DateTimeFeatureExtractor', DateTimeFeatureExtractor()),
    ('HolidayFeatureGenerator', HolidayFeatureGenerator()),
    ('TemporalFeatureEngineer',TemporalFeatureEngineer()),
    ('value_fill', NaFiller()),
    ('cat_encoder', CategoryMapper()),
    ('StoreDataProcessor', StoreDataProcessor())
])

In [37]:
train_dict = pipeline.fit_transform(train)


# Training

In [38]:
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [39]:
!pip install darts

  Using cached scikit_learn-1.7.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.7.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.7 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.1 which is incompatible.


In [40]:
!pip install scikit-learn==1.3.2


  Using cached scikit_learn-1.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.9 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.1
    Uninstalling scikit-learn-1.7.1:
      Successfully uninstalled scikit-learn-1.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
darts 0.36.0 requires scikit-learn>=1.6.0, but you have scikit-learn 1.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


In [41]:
from darts import TimeSeries
from darts.models.forecasting.arima import ARIMA

def forecast_store_models(p, d, q, num_train_weeks):
    error_metrics = []
    fitted_models = {}
    
    for store_id in train_dict.keys():
        #get store data
        df, metadata = train_dict[store_id]
    
        #Create timeseries object
        series_full = TimeSeries.from_dataframe(df,time_col='Date',value_cols=['Weekly_Sales'])
        series_train = series_full[:num_train_weeks]
        series_val = series_full[num_train_weeks:]
    
        #Fit and validate model
        model = ARIMA(p=p,d=d,q=q)
        model.fit(series_train)
        forecast = model.predict(len(series_val))
        
        # Calculate error metric
        actual_vals = series_val.values().flatten()
        pred_vals = forecast.values().flatten()
        mask = actual_vals != 0
        mape = np.mean(np.abs((actual_vals[mask] - pred_vals[mask]) / actual_vals[mask])) * 100
        error_metrics.append(mape)

        #Store fitted model
        fitted_models[store_id] = (model, metadata)
        
    return (error_metrics, fitted_models)

In [43]:
from tqdm import tqdm
param_list = [
        [2,1,1], 
        [2,1,0],
        [2,0,1],
        [2,2,1],
        [2,1,2],
        [1,1,1],
        [1,0,1],
        [1,1,0],
        [1,2,1],
        [1,1,2],
        [3,1,1],
        [3,0,1],
        [3,1,0],
        [3,2,1],
        [3,1,2],
        [0,1,1],
        [0,0,1],
        [0,1,0],
        [0,2,1],
        [0,1,2]
    ]

In [44]:
import dagshub
import mlflow
import pickle
import numpy as np
from datetime import datetime

# Initialize experiment tracking
mlflow.set_experiment("ARIMA_Hyperparameter_Study")

# Begin optimization tracking
with mlflow.start_run(run_name=f"ARIMA_Optimization_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
    
    # Record study settings
    mlflow.log_param("algorithm", "ARIMA")
    mlflow.log_param("test_period", 123)
    mlflow.log_param("search_space_size", len(param_list))
    mlflow.log_param("search_combinations", str(param_list))
    
    from tqdm import tqdm
    param_list = [
        [2,1,1], 
        [2,1,0],
        [2,0,1],
        [2,2,1],
        [2,1,2],
        [1,1,1],
        [1,0,1],
        [1,1,0],
        [1,2,1],
        [1,1,2],
        [3,1,1],
        [3,0,1],
        [3,1,0],
        [3,2,1],
        [3,1,2],
        [0,1,1],
        [0,0,1],
        [0,1,0],
        [0,2,1],
        [0,1,2]
    ]
    
    min_error = 100
    optimal_models = {}
    optimal_config = []
    
    for order in tqdm(param_list):
        # Track individual configurations
        with mlflow.start_run(run_name=f"ARIMA({order[0]},{order[1]},{order[2]})", nested=True):
            
            # Log configuration
            mlflow.log_param("ar_order", order[0])
            mlflow.log_param("diff_order", order[1]) 
            mlflow.log_param("ma_order", order[2])
            
            # Evaluate configuration
            error_metrics, fitted_models = forecast_store_models(order[0], order[1], order[2], 123)
            avg_error = sum(error_metrics)/len(error_metrics)
            
            # Record performance
            mlflow.log_metric("avg_error", avg_error)
            mlflow.log_metric("error_std", np.std(error_metrics))
            mlflow.log_metric("valid_stores", len(fitted_models))
            
            print(f'Average prediction error: {avg_error} for ARIMA({order[0]}, {order[1]}, {order[2]})')
            
            # Update if better performance found
            if avg_error < min_error:
                optimal_models = fitted_models
                min_error = avg_error
                optimal_config = [order[0], order[1], order[2]]
                
                mlflow.log_metric("is_optimal", 1)
            else:
                mlflow.log_metric("is_optimal", 0)
    
    # Record final optimal configuration
    mlflow.log_param("optimal_ar", optimal_config[0])
    mlflow.log_param("optimal_diff", optimal_config[1])
    mlflow.log_param("optimal_ma", optimal_config[2])
    mlflow.log_metric("best_avg_error", min_error)
    
    # Save optimal model set
    with open("optimal_arima_models.pkl", "wb") as f:
        pickle.dump(optimal_models, f)
    mlflow.log_artifact("optimal_arima_models.pkl")

2025/07/29 10:25:24 INFO mlflow.tracking.fluent: Experiment with name 'ARIMA_Hyperparameter_Study' does not exist. Creating a new experiment.
  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarnin

Average prediction error: 6.022589286896554 for ARIMA(2, 1, 1)
🏃 View run ARIMA(2,1,1) at: https://dagshub.com/TamariToradze/ML-Final.mlflow/#/experiments/11/runs/d198f243ed63471d9e66318f32cf7adb
🧪 View experiment at: https://dagshub.com/TamariToradze/ML-Final.mlflow/#/experiments/11


  5%|▌         | 1/20 [00:13<04:22, 13.80s/it]

Average prediction error: 7.178033536595267 for ARIMA(2, 1, 0)
🏃 View run ARIMA(2,1,0) at: https://dagshub.com/TamariToradze/ML-Final.mlflow/#/experiments/11/runs/f329a943087d4c419adafaca9aed87eb
🧪 View experiment at: https://dagshub.com/TamariToradze/ML-Final.mlflow/#/experiments/11


 10%|█         | 2/20 [00:31<04:52, 16.27s/it]/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters foun

...

Average prediction error: 16.00462855644804 for ARIMA(0, 2, 1)
🏃 View run ARIMA(0,2,1) at: https://dagshub.com/TamariToradze/ML-Final.mlflow/#/experiments/11/runs/a7ebe88faad1484a8ccb857a9df478de
🧪 View experiment at: https://dagshub.com/TamariToradze/ML-Final.mlflow/#/experiments/11


 95%|█████████▌| 19/20 [05:37<00:18, 18.00s/it]/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


Average prediction error: 6.52880753605619 for ARIMA(0, 1, 2)
🏃 View run ARIMA(0,1,2) at: https://dagshub.com/TamariToradze/ML-Final.mlflow/#/experiments/11/runs/aff67fbc2f51491b92f92fca790dfda5
🧪 View experiment at: https://dagshub.com/TamariToradze/ML-Final.mlflow/#/experiments/11


100%|██████████| 20/20 [05:55<00:00, 17.79s/it]


🏃 View run ARIMA_Optimization_20250729_102524 at: https://dagshub.com/TamariToradze/ML-Final.mlflow/#/experiments/11/runs/a8d1de243b3f4e16ace4185bf1005217
🧪 View experiment at: https://dagshub.com/TamariToradze/ML-Final.mlflow/#/experiments/11


# Predictions

In [45]:
forecast_results = {}
training_cutoff = pd.to_datetime(train['Date'].max())

for idx, row in tqdm(test.iterrows()):
    store_id = row['Store']
    dept_id = row['Dept']
    target_date = pd.to_datetime(row['Date'])
    forecast_horizon = (target_date - training_cutoff).days // 7

    fitted_model, dept_weights = optimal_models[store_id]
    store_forecast = fitted_model.predict(forecast_horizon).values()[-1, 0]
    
    if dept_id in dept_weights.keys():
        dept_forecast = store_forecast * dept_weights[dept_id]
    else:
        dept_forecast = store_forecast / len(dept_weights)
        
    forecast_results[(store_id, dept_id, target_date)] = dept_forecast

115064it [06:41, 286.31it/s]


In [46]:
forecast_df = pd.DataFrame([
    {
        'Id': f"{store_id}_{dept_id}_{target_date.strftime('%Y-%m-%d')}",
        'Weekly_Sales': sales_prediction
    }
    for (store_id, dept_id, target_date), sales_prediction in forecast_results.items()
])

forecast_df.to_csv('arima_predictions.csv', index=False)

In [47]:
import mlflow
import pandas as pd
from mlflow.tracking import MlflowClient

# Initialize MLflow client
client = MlflowClient()

# Get the experiment
experiment = mlflow.get_experiment_by_name("ARIMA_Parameter_Optimization")
experiment_id = experiment.experiment_id

print(f"Experiment ID: {experiment_id}")
print("="*80)

# Get all runs from this experiment
runs = mlflow.search_runs(experiment_ids=[experiment_id])

# Separate parent runs from child runs
parent_runs = runs[runs['tags.mlflow.parentRunId'].isna()]
child_runs = runs[runs['tags.mlflow.parentRunId'].notna()]

print("\n📊 PARENT RUN SUMMARY (Overall Grid Search)")
print("="*80)

for _, parent_run in parent_runs.iterrows():
    print(f"Run ID: {parent_run['run_id']}")
    print(f"Run Name: {parent_run['tags.mlflow.runName']}")
    print(f"Status: {parent_run['status']}")
    print(f"Start Time: {parent_run['start_time']}")
    print(f"End Time: {parent_run['end_time']}")
    
    print("\n📋 CONFIGURATION PARAMETERS:")
    print(f"  • Model Type: {parent_run.get('params.model_type', 'N/A')}")
    print(f"  • Validation Weeks: {parent_run.get('params.validation_weeks', 'N/A')}")
    print(f"  • Total Parameter Combinations: {parent_run.get('params.total_param_combinations', 'N/A')}")
    
    print("\n🏆 BEST MODEL RESULTS:")
    print(f"  • Best Parameters: ARIMA({parent_run.get('params.best_p', 'N/A')}, {parent_run.get('params.best_d', 'N/A')}, {parent_run.get('params.best_q', 'N/A')})")
    print(f"  • Best Mean MAPE: {parent_run.get('metrics.best_mean_mape', 'N/A'):.4f}")

print("\n\n📈 INDIVIDUAL PARAMETER COMBINATION RESULTS")
print("="*80)

# Sort child runs by mean_mape for better readability
child_runs_sorted = child_runs.sort_values('metrics.mean_mape', ascending=True)

print(f"{'Rank':<5} {'ARIMA Params':<15} {'Mean MAPE':<12} {'MAPE Std':<12} {'Stores':<8} {'Best?':<6}")
print("-" * 70)

for idx, (_, run) in enumerate(child_runs_sorted.iterrows(), 1):
    p = run.get('params.p', 'N/A')
    d = run.get('params.d', 'N/A') 
    q = run.get('params.q', 'N/A')
    mean_mape = run.get('metrics.mean_mape', float('inf'))
    mape_std = run.get('metrics.mape_std', 0)
    num_stores = run.get('metrics.num_successful_stores', 0)
    is_best = "✓" if run.get('metrics.is_best_model', 0) == 1 else "✗"
    
    print(f"{idx:<5} ARIMA({p},{d},{q}){'':<4} {mean_mape:<12.4f} {mape_std:<12.4f} {num_stores:<8.0f} {is_best:<6}")

print("\n\n📊 DETAILED STATISTICS")
print("="*80)

# Calculate overall statistics
all_mapes = child_runs_sorted['metrics.mean_mape'].dropna()
all_stores = child_runs_sorted['metrics.num_successful_stores'].dropna()

print(f"Total Parameter Combinations Tested: {len(child_runs_sorted)}")
print(f"Mean MAPE Statistics:")
print(f"  • Overall Best MAPE: {all_mapes.min():.4f}")
print(f"  • Overall Worst MAPE: {all_mapes.max():.4f}")
print(f"  • Average MAPE across all combinations: {all_mapes.mean():.4f}")
print(f"  • MAPE Standard Deviation: {all_mapes.std():.4f}")
print(f"\nStore Coverage:")
print(f"  • Average stores per parameter combination: {all_stores.mean():.1f}")
print(f"  • Min stores covered: {all_stores.min():.0f}")
print(f"  • Max stores covered: {all_stores.max():.0f}")

print("\n\n🔍 TOP 3 PERFORMING MODELS")
print("="*80)

top_3 = child_runs_sorted.head(3)
for idx, (_, run) in enumerate(top_3.iterrows(), 1):
    p = run.get('params.p', 'N/A')
    d = run.get('params.d', 'N/A')
    q = run.get('params.q', 'N/A')
    mean_mape = run.get('metrics.mean_mape', 'N/A')
    mape_std = run.get('metrics.mape_std', 'N/A')
    num_stores = run.get('metrics.num_successful_stores', 'N/A')
    
    print(f"{idx}. ARIMA({p},{d},{q})")
    print(f"   Mean MAPE: {mean_mape:.4f}")
    print(f"   MAPE Std: {mape_std:.4f}")
    print(f"   Successful Stores: {num_stores:.0f}")
    print()

print("\n\n📁 ARTIFACTS")
print("="*80)

# Check for artifacts in the parent run
for _, parent_run in parent_runs.iterrows():
    run_id = parent_run['run_id']
    artifacts = client.list_artifacts(run_id)
    if artifacts:
        print("Saved Artifacts:")
        for artifact in artifacts:
            print(f"  • {artifact.path}")
    else:
        print("No artifacts found")

print("\n" + "="*80)
print("✅ Metrics summary complete!")

Experiment ID: 10

📊 PARENT RUN SUMMARY (Overall Grid Search)
Run ID: 24a78d2f732b4cfdb756e3638649d682
Run Name: ARIMA_Grid_Search_20250729_095429
Status: FINISHED
Start Time: 2025-07-29 09:54:30.199000+00:00
End Time: 2025-07-29 09:57:33.780000+00:00

📋 CONFIGURATION PARAMETERS:
  • Model Type: ARIMA
  • Validation Weeks: 123
  • Total Parameter Combinations: 10

🏆 BEST MODEL RESULTS:
  • Best Parameters: ARIMA(2, 1, 1)
  • Best Mean MAPE: 6.0226
Run ID: 3036eba13aec4a21aff2921ab015c587
Run Name: ARIMA_Grid_Search_20250728_230242
Status: FINISHED
Start Time: 2025-07-28 23:02:42.736000+00:00
End Time: 2025-07-28 23:05:46.356000+00:00

📋 CONFIGURATION PARAMETERS:
  • Model Type: ARIMA
  • Validation Weeks: 123
  • Total Parameter Combinations: 10

🏆 BEST MODEL RESULTS:
  • Best Parameters: ARIMA(2, 1, 1)
  • Best Mean MAPE: 6.0226
Run ID: dcbaa8472ff5489e9fb9b447a2d1dcc7
Run Name: ARIMA_Grid_Search_20250728_225140
Status: RUNNING
Start Time: 2025-07-28 22:51:40.818000+00:00
End Time: Na